In [2]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
from scipy.stats import gaussian_kde
from astropy.cosmology import Planck18 as cosmo
from scipy.spatial import cKDTree

import matplotlib
from matplotlib import pyplot as plt

In [3]:
quaia_sdss_file = "Matched_Quaia_SDSS_Catalog.fits"
desi_file = "DESI_QSO_only.fits"
output_file = "All_Matched_Catalog.fits"

quaia_sdss = fits.open(quaia_sdss_file)[1].data
desi = fits.open(desi_file)[1].data

print(len(quaia_sdss))

218547


In [4]:
quaia_sdss_coords = SkyCoord(ra=quaia_sdss['Quaia_RA']*u.deg, dec=quaia_sdss['Quaia_DEC']*u.deg)
desi_coords = SkyCoord(ra=desi['TARGET_RA']*u.deg, dec=desi['TARGET_DEC']*u.deg)

idx, d2d, _ = quaia_sdss_coords.match_to_catalog_sky(desi_coords)
matched = d2d < 1 * u.arcsec #1 arcsec

#NaN values
# Get matched indices
quaia_matched_idx = np.where(matched)[0]
desi_matched_idx = idx[matched]

# Prepare new columns with NaN placeholders
desi_targetid = np.full(len(quaia_sdss), -1, dtype=int)
desi_z = np.full(len(quaia_sdss), np.nan)
desi_ra = np.full(len(quaia_sdss), np.nan)
desi_dec = np.full(len(quaia_sdss), np.nan)
desi_flux_g = np.full(len(quaia_sdss), np.nan)
desi_flux_r = np.full(len(quaia_sdss), np.nan)
desi_flux_z = np.full(len(quaia_sdss), np.nan)
desi_flux_w1 = np.full(len(quaia_sdss), np.nan)
desi_flux_w2 = np.full(len(quaia_sdss), np.nan)

In [5]:
desi_targetid[quaia_matched_idx] = desi['TARGETID'][desi_matched_idx]
desi_z[quaia_matched_idx] = desi['Z'][desi_matched_idx]
desi_ra[quaia_matched_idx] = desi['TARGET_RA'][desi_matched_idx]
desi_dec[quaia_matched_idx] = desi['TARGET_DEC'][desi_matched_idx]
desi_flux_g[quaia_matched_idx] = desi['FLUX_G'][desi_matched_idx]
desi_flux_r[quaia_matched_idx] = desi['FLUX_R'][desi_matched_idx]
desi_flux_z[quaia_matched_idx] = desi['FLUX_Z'][desi_matched_idx]
desi_flux_w1[quaia_matched_idx] = desi['FLUX_W1'][desi_matched_idx]
desi_flux_w2[quaia_matched_idx] = desi['FLUX_W2'][desi_matched_idx]

new_cols = [
    fits.Column(name='DESI_TARGETID', format='K', array=desi_targetid),
    fits.Column(name='DESI_Z', format='D', array=desi_z),
    fits.Column(name='DESI_TARGET_RA', format='D', array=desi_ra),
    fits.Column(name='DESI_TARGET_DEC', format='D', array=desi_dec),
    fits.Column(name='DESI_FLUX_G', format='E', array=desi_flux_g),
    fits.Column(name='DESI_FLUX_R', format='E', array=desi_flux_r),
    fits.Column(name='DESI_FLUX_Z', format='E', array=desi_flux_z),
    fits.Column(name='DESI_FLUX_W1', format='E', array=desi_flux_w1),
    fits.Column(name='DESI_FLUX_W2', format='E', array=desi_flux_w2)
]

In [6]:
orig_columns = fits.ColDefs(quaia_sdss)
new_coldefs = fits.ColDefs(new_cols)
merged_table = fits.BinTableHDU.from_columns(orig_columns + new_coldefs)

# Save the new catalog
#merged_table.writeto(output_file, overwrite=True)
print(f"Matched catalog saved to {output_file}")

Matched catalog saved to All_Matched_Catalog.fits


In [7]:
all = fits.open('All_Matched_Catalog.fits')

all[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  166 / length of dimension 1                          
NAXIS2  =               218547 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   25 / number of table fields                         
TTYPE1  = 'Quaia_id'                                                            
TFORM1  = 'K       '                                                            
TTYPE2  = 'Quaia_RA'                                                            
TFORM2  = 'D       '                                                            
TTYPE3  = 'Quaia_DEC'       

## DESI DR1

In [8]:
desidr1 = fits.open('QSO_cat_iron_cumulative_v0.fits')
quaia_fits = fits.open('quaia_G20.5.fits')

desidr1_data = desidr1[1].data
quaia_data = quaia_fits[1].data

In [9]:
desidr1[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  374 / length of dimension 1                          
NAXIS2  =              2182309 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   68 / number of table fields                         
EXTNAME = 'QSO_CAT '           / name of this binary table extension            
DESIDR  = 'dr1     '                                                            
TTYPE1  = 'TARGETID'                                                            
TFORM1  = 'K       '                                                            
TTYPE2  = 'Z       '        

In [10]:
desidr1_ra, desidr1_dec, desidr1_z = desidr1_data['TARGET_RA'], desidr1_data['TARGET_DEC'], desidr1_data['Z']
quaia_z, quaia_ra, quaia_dec = quaia_data['redshift_quaia'], quaia_data['ra'], quaia_data['dec']

desi_coords = SkyCoord(ra=desidr1_data['TARGET_RA']*u.deg, dec=desidr1_data['TARGET_DEC']*u.deg)
quaia_coords = SkyCoord(ra=quaia_ra*u.deg, dec=quaia_dec*u.deg)

idx, d2d, _ = quaia_coords.match_to_catalog_sky(desi_coords)
matched = d2d < 1 * u.arcsec #1 arcsec

quaia_idx = np.where(matched)[0]
desi_idx = idx[matched]

In [11]:
# Prepare new columns with NaN placeholders
desi_z = np.full(len(quaia_data), np.nan)
desi_ra = np.full(len(quaia_data), np.nan)
desi_dec = np.full(len(quaia_data), np.nan)
desi_flux_g = np.full(len(quaia_data), np.nan)
desi_flux_r = np.full(len(quaia_data), np.nan)
desi_flux_z = np.full(len(quaia_data), np.nan)
desi_flux_w1 = np.full(len(quaia_data), np.nan)
desi_flux_w2 = np.full(len(quaia_data), np.nan)

print(len(quaia_z), len(desi_z))

1295502 1295502


In [12]:
desi_z[quaia_idx] = desidr1_data['Z'][desi_idx]
desi_ra[quaia_idx] = desidr1_data['TARGET_RA'][desi_idx]
desi_dec[quaia_idx] = desidr1_data['TARGET_DEC'][desi_idx]
desi_flux_g[quaia_idx] = desidr1_data['FLUX_G'][desi_idx]
desi_flux_r[quaia_idx] = desidr1_data['FLUX_R'][desi_idx]
desi_flux_z[quaia_idx] = desidr1_data['FLUX_Z'][desi_idx]
desi_flux_w1[quaia_idx] = desidr1_data['FLUX_W1'][desi_idx]
desi_flux_w2[quaia_idx] = desidr1_data['FLUX_W2'][desi_idx]

new_cols = [
    fits.Column(name='DESI_Z', format='D', array=desi_z),
    fits.Column(name='DESI_TARGET_RA', format='D', array=desi_ra),
    fits.Column(name='DESI_TARGET_DEC', format='D', array=desi_dec),
    fits.Column(name='DESI_FLUX_G', format='E', array=desi_flux_g),
    fits.Column(name='DESI_FLUX_R', format='E', array=desi_flux_r),
    fits.Column(name='DESI_FLUX_Z', format='E', array=desi_flux_z),
    fits.Column(name='DESI_FLUX_W1', format='E', array=desi_flux_w1),
    fits.Column(name='DESI_FLUX_W2', format='E', array=desi_flux_w2),
    
    fits.Column(name='Quaia_RA', format='D', array=quaia_ra[quaia_idx]),
    fits.Column(name='Quaia_DEC', format='D', array=quaia_dec[quaia_idx]),
    fits.Column(name='Quaia_Z', format='D', array=quaia_z[quaia_idx]),
    fits.Column(name='Quaia_G_Mean_MAG', format='E', array=quaia_data['phot_g_mean_mag'][quaia_idx]),
    fits.Column(name='Quaia_BP_Mean_MAG', format='E', array=quaia_data['phot_bp_mean_mag'][quaia_idx]),
    fits.Column(name='Quaia_RP_Mean_MAG', format='E', array=quaia_data['phot_rp_mean_mag'][quaia_idx]),
    fits.Column(name='Quaia_W1_MAG', format='D', array=quaia_data['mag_w1_vg'][quaia_idx]),
    fits.Column(name='Quaia_W2_MAG', format='D', array=quaia_data['mag_w2_vg'][quaia_idx])
]

In [13]:
orig_columns = fits.ColDefs(quaia_data)
new_coldefs = fits.ColDefs(new_cols)
merged_table = fits.BinTableHDU.from_columns(orig_columns + new_coldefs)

# Save the new catalog
output_file1 = 'Quaia_DESI_DR1.fits'

#merged_table.writeto(output_file1, overwrite=True)
print(f"Matched catalog saved to {output_file1}")

Matched catalog saved to Quaia_DESI_DR1.fits


In [14]:
desiqso = fits.open('agnqso_desi.fits')
desiqso_data = desiqso[1].data

desiqso_only = desiqso_data[desiqso_data['SPECTYPE']=='QSO']

print(len(desiqso_only))

1772694


In [15]:
#agnqso_desi.fits file id matching to QSO_cat_iron_cumulative_v0.fits for the QSO only catalogue

desi_qso_id = desiqso_only['TARGETID']
desidr1_id = desidr1_data['TARGETID']

matched_id = np.isin(desidr1_id, desi_qso_id)
filtered_qso = desidr1_data[matched_id]

hdu = fits.BinTableHDU(data=filtered_qso, header=desidr1[1].header)
#hdu.writeto("DESI_qso_matched.fits", overwrite=True)



In [16]:
print(len(filtered_qso))

2108273


quaia_data = quaia_fits[1].data

quaia_z, quaia_ra, quaia_dec = quaia_data['redshift_quaia'], quaia_data['ra'], quaia_data['dec']

quaia_coords = SkyCoord(ra=quaia_ra*u.deg, dec=quaia_dec*u.deg)

    fits.Column(name='Quaia_RA', format='D', array=quaia_ra[quaia_idx]),
    fits.Column(name='Quaia_DEC', format='D', array=quaia_dec[quaia_idx]),
    fits.Column(name='Quaia_Z', format='D', array=quaia_z[quaia_idx]),
    fits.Column(name='Quaia_G_Mean_MAG', format='E', array=quaia_data['phot_g_mean_mag'][quaia_idx]),
    fits.Column(name='Quaia_BP_Mean_MAG', format='E', array=quaia_data['phot_bp_mean_mag'][quaia_idx]),
    fits.Column(name='Quaia_RP_Mean_MAG', format='E', array=quaia_data['phot_rp_mean_mag'][quaia_idx]),
    fits.Column(name='Quaia_W1_MAG', format='D', array=quaia_data['mag_w1_vg'][quaia_idx]),
    fits.Column(name='Quaia_W2_MAG', format='D', array=quaia_data['mag_w2_vg'][quaia_idx])

## SUPERSET FITS File

In [17]:
qsoonly = fits.open("DESI_qso_matched.fits")
qsoonly_data = qsoonly[1].data

sdss = fits.open("SDSSDR16Q_QSO_only.fits")
sdss_data = sdss[1].data

desiqso_z, desiqso_ra, desiqso_dec = qsoonly_data['Z'], qsoonly_data['TARGET_RA'], qsoonly_data['TARGET_DEC']
sdss_z, sdss_ra, sdss_dec = sdss_data['Z'], sdss_data['RA'], sdss_data['DEC']

desiqso_coords = SkyCoord(ra=desiqso_ra*u.deg, dec=desiqso_dec*u.deg)
sdss_coords = SkyCoord(ra=sdss_ra*u.deg, dec=sdss_dec*u.deg)

idx, d2d, _ = quaia_coords.match_to_catalog_sky(desiqso_coords)
matched2 = d2d < 1 * u.arcsec #1 arcsec

idxx, d2dx, _ = quaia_coords.match_to_catalog_sky(sdss_coords)
matched3 = d2dx < 1 * u.arcsec #1 arcsec

quaia_idx2 = np.where(matched2)[0]
desiqso_idx = idx[matched2]
quaia_idxx = np.where(matched3)[0]
sdss_idxx = idxx[matched3]

In [18]:
sdss[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 2290 / length of dimension 1                          
NAXIS2  =               624650 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  183 / number of table fields                         
EXTNAME = 'CATALOG '           / extension name                                 
TTYPE1  = 'SDSS_NAME'                                                           
TFORM1  = '18A     '                                                            
TTYPE2  = 'RA      '                                                            
TFORM2  = 'D       '        

In [19]:
desiqso_z = np.full(len(quaia_data), np.nan)
desiqso_ra = np.full(len(quaia_data), np.nan)
desiqso_dec = np.full(len(quaia_data), np.nan)
desiqso_flux_g = np.full(len(quaia_data), np.nan)
desiqso_flux_r = np.full(len(quaia_data), np.nan)
desiqso_flux_z = np.full(len(quaia_data), np.nan)
desiqso_flux_w1 = np.full(len(quaia_data), np.nan)
desiqso_flux_w2 = np.full(len(quaia_data), np.nan)

#desiqso_g = np.full(len(quaia_data), np.nan)
#desiqso_r = np.full(len(quaia_data), np.nan)
#desiqso_z_mag = np.full(len(quaia_data), np.nan)
#desiqso_w1 = np.full(len(quaia_data), np.nan)
#desiqso_w2 = np.full(len(quaia_data), np.nan)

sdss_z = np.full(len(quaia_data), np.nan)
sdss_ra = np.full(len(quaia_data), np.nan)
sdss_dec = np.full(len(quaia_data), np.nan)
sdss_g = np.full(len(quaia_data), np.nan)
sdss_w1 = np.full(len(quaia_data), np.nan)
sdss_w2 = np.full(len(quaia_data), np.nan)

#quaia_w12 = np.full(len(quaia_data), np.nan)

print(len(quaia_z), len(desiqso_z), len(sdss_z))

1295502 1295502 1295502


In [20]:
def flux_to_mag(flux):
    with np.errstate(divide='ignore', invalid='ignore'):  # log(0) 에러 방지
        mag = 22.5 - 2.5 * np.log10(flux)
        mag[flux <= 0] = np.nan  # 0 이하 값은 NaN 처리
    return mag

In [25]:
desiqso_z[quaia_idx2] = qsoonly_data['Z'][desiqso_idx]
desiqso_ra[quaia_idx2] = qsoonly_data['TARGET_RA'][desiqso_idx]
desiqso_dec[quaia_idx2] = qsoonly_data['TARGET_DEC'][desiqso_idx]
desiqso_flux_g[quaia_idx2] = qsoonly_data['FLUX_G'][desiqso_idx]
desiqso_flux_r[quaia_idx2] = qsoonly_data['FLUX_R'][desiqso_idx]
desiqso_flux_z[quaia_idx2] = qsoonly_data['FLUX_Z'][desiqso_idx]
desiqso_flux_w1[quaia_idx2] = qsoonly_data['FLUX_W1'][desiqso_idx]
desiqso_flux_w2[quaia_idx2] = qsoonly_data['FLUX_W2'][desiqso_idx]

desiqso_g = flux_to_mag(desiqso_flux_g)
desiqso_r = flux_to_mag(desiqso_flux_r)
desiqso_z_mag = flux_to_mag(desiqso_flux_z)
desiqso_w1 = flux_to_mag(desiqso_flux_w1)
desiqso_w2 = flux_to_mag(desiqso_flux_w2)

sdss_z[quaia_idxx] = sdss_data['Z'][sdss_idxx]
sdss_ra[quaia_idxx] = sdss_data['RA'][sdss_idxx]
sdss_dec[quaia_idxx] = sdss_data['DEC'][sdss_idxx]
sdss_g[quaia_idxx] = sdss_data['GAIA_G_MAG'][sdss_idxx]
sdss_w1[quaia_idxx] = sdss_data['W1_MAG'][sdss_idxx]
sdss_w2[quaia_idxx] = sdss_data['W2_MAG'][sdss_idxx]

#quaia mag w1-w2
quaia_w12 = quaia_data['mag_w1_vg'] - quaia_data['mag_w2_vg']

new_cols = [
    fits.Column(name='DESI_Z', format='D', array=desiqso_z),
    fits.Column(name='DESI_TARGET_RA', format='D', array=desiqso_ra),
    fits.Column(name='DESI_TARGET_DEC', format='D', array=desiqso_dec),
    fits.Column(name='DESI_FLUX_G', format='E', array=desiqso_flux_g),
    fits.Column(name='DESI_FLUX_R', format='E', array=desiqso_flux_r),
    fits.Column(name='DESI_FLUX_Z', format='E', array=desiqso_flux_z),
    fits.Column(name='DESI_FLUX_W1', format='E', array=desiqso_flux_w1),
    fits.Column(name='DESI_FLUX_W2', format='E', array=desiqso_flux_w2),

    fits.Column(name='DESI_g_MAG', format='E', array=desiqso_g),
    fits.Column(name='DESI_r_MAG', format='E', array=desiqso_r),
    fits.Column(name='DESI_z_MAG', format='E', array=desiqso_z_mag),
    fits.Column(name='DESI_w1_MAG', format='E', array=desiqso_w1),
    fits.Column(name='DESI_w2_MAG', format='E', array=desiqso_w2),


    fits.Column(name='SDSS_RA', format='D', array=sdss_ra),
    fits.Column(name='SDSS_DEC', format='D', array=sdss_dec),
    fits.Column(name='SDSS_Z', format='D', array=sdss_z),
    fits.Column(name='SDSS_G_MAG', format='E', array=sdss_g),
    fits.Column(name='SDSS_W1_MAG', format='E', array=sdss_w1),
    fits.Column(name='SDSS_W2_MAG', format='E', array=sdss_w2),

    fits.Column(name='Quaia_W12', format='E', array=quaia_w12)
]

In [26]:
orig_columns = fits.ColDefs(quaia_data)
new_coldefs = fits.ColDefs(new_cols)
merged_table = fits.BinTableHDU.from_columns(orig_columns + new_coldefs)

# Save the new catalog
output_file2 = 'Quaia_DESIQSO_DR1_SDSS.fits'

merged_table.writeto(output_file2, overwrite=True)
print(f"Matched catalog saved to {output_file2}")

Matched catalog saved to Quaia_DESIQSO_DR1_SDSS.fits


In [27]:
superset = fits.open('Quaia_DESIQSO_DR1_SDSS.fits')
superset_data = superset[1].data

numdesi = superset_data[np.isfinite(superset_data['DESI_Z'])]
numsdss = superset_data[np.isfinite(superset_data['SDSS_Z'])]
numquaia = superset_data[np.isfinite(superset_data['redshift_quaia'])]

print(len(numdesi), len(numsdss), len(numquaia))

numforall = superset_data[np.isfinite(superset_data['DESI_Z']) & np.isfinite(superset_data['SDSS_Z'])]

print(len(numforall))

#이거 하나하나 세려면 어떻게 하더라? 기본적으로 quaia 베이스라서 뭘 같이 넣어도 quaia 랑 겹치게 되는건데..
#따로 SDSS n DESI 이렇게 셀 수 있게 하는거 찾아야함.

314807 218546 1295502
101233


In [28]:
superset[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  236 / length of dimension 1                          
NAXIS2  =              1295502 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   38 / number of table fields                         
TTYPE1  = 'source_id'                                                           
TFORM1  = 'K       '                                                            
TNULL1  =               999999                                                  
TTYPE2  = 'unwise_objid'                                                        
TFORM2  = '16A     '        